In [ ]:
import os, math
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.datasets import cifar10, mnist
import matplotlib.pyplot as plt
import random
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization 
from tensorflow.keras.layers import Conv2D, Flatten, Activation
from tensorflow.keras.layers import Reshape, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import mse, SparseCategoricalCrossentropy

In [ ]:
(x_train, y_train),(x_test, _) = mnist.load_data()


11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1)


In [ ]:
image_size = x_train[0].shape[1]
image_size

28

In [ ]:
class ConvTransBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides):
        super().__init__()
        self.bn = BatchNormalization()  # check training flag
        self.act = Activation(activation='relu')
        self.conv2D_trans = Conv2DTranspose(filters=filters,
                                kernel_size=kernel_size,
                                strides=strides,
                                padding='same')
        
    def call(self, inputs, training=False):
        x = self.bn(inputs)
        x = self.act(x)
        return self.conv2D_trans(x)


In [ ]:
noise = np.random.uniform(0, 1, size=[10000, 100])


In [ ]:
model = Sequential()
model.add(Input(noise.shape[1]))
model.add(Dense(14*14*32, activation="relu"))
model.add(Reshape([14, 14, 32]))
model.add(ConvTransBlock(32, 5, 2))
model.add(ConvTransBlock(16, 5, 1))
model.add(Conv2DTranspose(1, 5, padding="same", activation="sigmoid"))


In [ ]:
model.compile(loss="MSE", metrics=["MAE"])
model.fit(noise, x_train.reshape(-1, 28, 28, 1)[:10000])

In [ ]:
class Generator(tf.keras.models.Model):
    def __init__(self, filters, kernel_size, resize_img):
        super().__init__()
        self.dense1 = Dense(resize_img * resize_img * filters[0])
        self.reshape = Reshape([resize_img, resize_img, filters[0]])
        self.conv2dtrans = []
        for i, _filter in enumerate(filters):
            if i <= 1:
                strides = 2
            else:
                strides = 1
            self.conv2dtrans.append(ConvTransBlock(_filter, kernel_size, strides))
        
        self.act = Activation("sigmoid")

    def call(self, inputs, training=False):
        x = self.dense1(inputs)
        x = self.reshape(x)
        for conv in self.conv2dtrans:
            x = conv(x)
        return self.act(x)




In [ ]:
gen = Generator(gen_layers_filter, gen_kernel_size, gen_resize_img)

NameError: ignored

In [ ]:
gen.compile(loss="mse", metrics=["mae"])

In [ ]:
gen.fit(noise, x_train.reshape(-1, 28, 28, 1)[:10000], batch_size=64)

In [ ]:
def leaky_conv(filters, kernel_size, strides):
    return Sequential([LeakyReLU(alpha=0.2),
                       Conv2D(filters=filters,
                       kernel_size=kernel_size,
                       strides=strides,
                       padding='same')])


In [ ]:
model = Sequential()
model.add(Input(x_train.shape[1:]))
model.add(leaky_conv(32, 5, 2))
model.add(leaky_conv(64, 5, 2))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation("softmax"))

In [ ]:
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_10 (Sequential)   (None, 14, 14, 32)        832       
_________________________________________________________________
sequential_11 (Sequential)   (None, 7, 7, 64)          51264     
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                31370     
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 83,466
Trainable params: 83,466
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train)

In [ ]:
class Discriminator(tf.keras.models.Model):
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.leaky_convs = []
        for i, _filter in enumerate(filters):
            if i < len(filters) - 1:
                strides = 2
            else:
                strides = 1
            self.leaky_convs.append(leaky_conv(_filter, kernel_size, strides))
        self.flat = Flatten()
        self.dense = Dense(1, activation='sigmoid')

    def call(self, inputs, training=False):
        x = inputs
        for conv in self.leaky_convs:
            x = conv(x)
        x = self.flat(x)
        return self.dense(x)

In [ ]:
disc = Discriminator(disc_layers_filters, disc_kernel_size)

In [ ]:
disc.compile(loss="categorical_crossentropy", metrics=["mae"])

In [ ]:
disc.fit(x_train, np.where(y_train < 5, 1, 0))

In [ ]:
disc.summary()

Model: "discriminator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_12 (Sequential)   (32, 14, 14, 32)          832       
_________________________________________________________________
sequential_13 (Sequential)   (32, 7, 7, 64)            51264     
_________________________________________________________________
sequential_14 (Sequential)   (32, 4, 4, 128)           204928    
_________________________________________________________________
sequential_15 (Sequential)   (32, 4, 4, 256)           819456    
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  4097      
Total params: 1,080,577
Trainable params: 1,080,577
Non-trainable params: 0
_________________________________________

In [ ]:
latent_size = 100
batch_size = 64
train_steps = 40000
disc_lr = 2e-4
disc_decay = 6e-8
gen_lr = disc_lr/2
gen_decay = disc_decay/2



In [ ]:
disc_kernel_size = 5
disc_layers_filters = [32, 64, 128, 256]

In [ ]:
gen_resize_img = image_size // 4
gen_kernel_size = 5
gen_layers_filter = [128, 64, 32, 1]

In [ ]:
gen = Generator(gen_layers_filter, gen_kernel_size, gen_resize_img)

In [ ]:
disc = Discriminator(disc_layers_filters, disc_kernel_size)
disc.compile(loss='binary_crossentropy',
             optimizer=RMSprop(learning_rate=disc_lr, decay=disc_decay),
             metrics=['accuracy'])


In [ ]:
def build_gan():
    # disc.trainable = False
    gan = Sequential(name='gan_model')
    gan.add(Input([100,]))
    gan.add(gen)
    gan.add(disc)
    gan.summary()
    gan.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=gen_lr, decay=gen_decay),
                metrics=['accuracy'])            
    return gan   


gan = build_gan()


Model: "gan_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator_1 (Generator)      (None, 28, 28, 1)         1301505   
_________________________________________________________________
discriminator_2 (Discriminat (None, 1)                 1080577   
Total params: 2,382,082
Trainable params: 2,381,378
Non-trainable params: 704
_________________________________________________________________


In [ ]:
for layer in gan.layers:
    print(layer.trainable)

True
False


In [ ]:
def plot_images(generator,
                noise_input,
                noise_label=None,
                noise_codes=None,
                show=False,
                step=0,
                model_name="gan"):
    """Generate fake images and plot them

    For visualization purposes, generate fake images
    then plot them in a square grid

    # Arguments
        generator (Model): The Generator Model for 
            fake images generation
        noise_input (ndarray): Array of z-vectors
        show (bool): Whether to show plot or not
        step (int): Appended to filename of the save images
        model_name (string): Model name

    """
    # os.makedirs(model_name, exist_ok=True)
    # filename = os.path.join(model_name, "%05d.png" % step)
    rows = int(math.sqrt(noise_input.shape[0]))
    if noise_label is not None:
        noise_input = [noise_input, noise_label]
        if noise_codes is not None:
            noise_input += noise_codes

    images = generator.predict(noise_input)
    plt.figure(figsize=(10, 10))
    num_images = images.shape[0]
    image_size = images.shape[1]
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    # plt.savefig(filename)
    if show:
        
        plt.show()
    else:
        plt.close('all')

In [ ]:
# Train the Discriminator and Adversarial Networks
def train(models, x_train):
    generator, discriminator, gan = models 
    # batch_size, latent_size, train_step, model_name = params
    save_interval = 500
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size]) 
    train_size = x_train.shape[0]

    for i in range(train_steps):
        rand_indexes = np.random.randint(0, train_size, size=batch_size)
        real_images = x_train[rand_indexes]
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        fake_images = generator.predict(noise)        
        x = np.concatenate((real_images, fake_images)) # same size
        y = np.ones([2 * batch_size, 1])
        y[batch_size:, :] = 0.0
        discriminator.trainable = True
        loss, acc = discriminator.train_on_batch(x, y)
        # log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)
        log = f'{i}: [discriminator loss: {loss}, acc: {acc}]'
        discriminator.trainable = False

        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        y = np.ones([batch_size, 1])        
        loss, acc = gan.train_on_batch(noise, y)
        log = "%s [gan loss: %f, acc: %f]" % (log, loss, acc)
        print(log)
        if (i + 1) % save_interval == 0:
            # plot generator images on a periodic basis
            plot_images(generator,
                        noise_input=noise_input,
                        show=True,
                        step=(i + 1),
                        model_name='gan_model')
    
        

In [ ]:
train(models=[gen, disc, gan], x_train=x_train)

0: [discriminator loss: 3.310608148574829, acc: 0.5390625] [gan loss: 0.442984, acc: 1.000000]
1: [discriminator loss: 0.759007453918457, acc: 0.5] [gan loss: 0.181721, acc: 1.000000]
2: [discriminator loss: 0.9960637092590332, acc: 0.5] [gan loss: 0.076730, acc: 1.000000]
3: [discriminator loss: 1.0921070575714111, acc: 0.5] [gan loss: 0.047926, acc: 1.000000]
4: [discriminator loss: 1.1062705516815186, acc: 0.5] [gan loss: 0.037410, acc: 1.000000]
5: [discriminator loss: 1.0864193439483643, acc: 0.5] [gan loss: 0.031826, acc: 1.000000]
6: [discriminator loss: 1.050199031829834, acc: 0.5] [gan loss: 0.027467, acc: 1.000000]
7: [discriminator loss: 1.0036964416503906, acc: 0.5] [gan loss: 0.024138, acc: 1.000000]
8: [discriminator loss: 0.9490150213241577, acc: 0.5] [gan loss: 0.021317, acc: 1.000000]
9: [discriminator loss: 0.8897984027862549, acc: 0.5] [gan loss: 0.019116, acc: 1.000000]
10: [discriminator loss: 0.8305274248123169, acc: 0.5] [gan loss: 0.017024, acc: 1.000000]
11: [d

NameError: ignored